In [110]:
import pandas as pd 
import nltk
import string

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [111]:
df = pd.read_csv("C:\\Users\\Eli\\Desktop\\train.csv", index_col=0) #index_col=0)

In [112]:
df

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0


In [113]:
df['text'][5]

'In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the wo

In [114]:
#df = df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   author  18843 non-null  object
 2   text    20761 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [120]:
df = df.dropna()

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18285 non-null  object
 1   author  18285 non-null  object
 2   text    18285 non-null  object
 3   label   18285 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [115]:
df['label'] = df['label'].replace({1:'Unreliable', 0:'reliable'}) 

In [122]:
#removing HTML data
def remove_html(text):
    soup = BeautifulSoup(text)
    html_free = soup.get_text()
    return html_free

In [123]:
df['text'] = df['text'].apply(lambda x: remove_html(x))
df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Unreliable
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,reliable
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Unreliable
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Unreliable
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Unreliable


In [124]:
def remove_punctuation(text):
    no_punctuation = "".join([c for c in text if c not in string.punctuation])
    return no_punctuation

In [126]:
df['text'] = df['text'].apply(lambda x: remove_punctuation(x))
df['text']

id
0        House Dem Aide We Didn’t Even See Comey’s Lett...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29 2...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T I unloaded on black celebrities who m...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO Russia To Hold Parallel Exercises In Balk...
20799    David Swanson is an author activist journalist...
Name: text, Length: 18285, dtype: object

In [127]:
tokenizer = RegexpTokenizer(r'\w+')

In [128]:
df['text'] = df['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
df['text'].head(20)

id
0     [house, dem, aide, we, didn, t, even, see, com...
1     [ever, get, the, feeling, your, life, circles,...
2     [why, the, truth, might, get, you, fired, octo...
3     [videos, 15, civilians, killed, in, single, us...
4     [print, an, iranian, woman, has, been, sentenc...
5     [in, these, trying, times, jackie, mason, is, ...
7     [paris, france, chose, an, idealistic, traditi...
9     [a, week, before, michael, t, flynn, resigned,...
10    [organizing, for, action, the, activist, group...
11    [the, bbc, produced, spoof, on, the, real, hou...
12    [the, mystery, surrounding, the, third, reich,...
13    [clinton, campaign, demands, fbi, affirm, trum...
14    [yes, there, are, paid, government, trolls, on...
15    [guillermo, barros, schelotto, was, not, the, ...
16    [the, scandal, engulfing, wells, fargo, topple...
17    [a, caddo, nation, tribal, leader, has, just, ...
18    [fbi, closes, in, on, hillary, posted, on, hom...
19    [wednesday, after, donald, trump, s, pr

In [129]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [131]:
df['text'] = df['text'].apply(lambda x: remove_stopwords(x))
df['text'].head(20)

id
0     [house, dem, aide, even, see, comey, letter, j...
1     [ever, get, feeling, life, circles, roundabout...
2     [truth, might, get, fired, october, 29, 2016, ...
3     [videos, 15, civilians, killed, single, us, ai...
4     [print, iranian, woman, sentenced, six, years,...
5     [trying, times, jackie, mason, voice, reason, ...
7     [paris, france, chose, idealistic, traditional...
9     [week, michael, flynn, resigned, national, sec...
10    [organizing, action, activist, group, morphed,...
11    [bbc, produced, spoof, real, housewives, tv, p...
12    [mystery, surrounding, third, reich, nazi, ger...
13    [clinton, campaign, demands, fbi, affirm, trum...
14    [yes, paid, government, trolls, social, media,...
15    [guillermo, barros, schelotto, first, argentin...
16    [scandal, engulfing, wells, fargo, toppled, ch...
17    [caddo, nation, tribal, leader, freed, spendin...
18    [fbi, closes, hillary, posted, home, headlines...
19    [wednesday, donald, trump, press, confe

In [132]:
stemmer = PorterStemmer()

In [133]:
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [134]:
df['text'] = df['text'].apply(lambda x: word_stemmer(x))
df['text'].tail(20)

id
20779    mani part world christian gather celebr christ...
20780    earli oct 6 erin awaken sound babi cri later t...
20781    brandon turbevil dea announc back f decis go e...
20782    home headlin world news fix nbc affili acciden...
20783    good morn need know south korean court announc...
20784    finian cunningham written extens intern affair...
20785    first sentenc congress obamacar repeal read sh...
20787    former deputi attorney gener salli yate declin...
20788    googl pinterest digg linkedin reddit stumbleup...
20789    senat major leader mitch mcconnel r ky recent ...
20790    u ambassador unit nation nikki haley blast new...
20791    lawyer kept hillari campaign chief jail doj hi...
20792    two suicid bomber attack bu station jakarta we...
20793    share although vandal thought cool destroy don...
20794    donald trump took twitter friday prais vladimi...
20795    rapper unload black celebr met donald trump el...
20796    green bay packer lost washington redskin wee

In [135]:
df.to_csv("C:\\Users\\Eli\\Desktop\\news_processed.csv")